# 📋 TensorBoard
[TensorBoard](https://tensorboard.dev) je nástroj, který se používá pro tvorbu vizualizací modelů strojového učení (neuronové sítě). Dokáže například sbírat data o vývoji nějaké metriky během trénování modelu a ty následně zobrazit v grafu, vizualizovat architekturu ML modelu a mnoho jiných věcí 🙂.

TensorBoard se vám může hodit například při bakalářské práci 👩🏽‍🎓, protože vzniklé vizualizace lze sdílet pomocí odkazu.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from datetime import datetime

V minulých noteboocích jsme si říkali, že příkazy, které začínají na %, se nazývají [magické příkazy](https://ipython.readthedocs.io/en/stable/interactive/magics.html). Tento spustí TensorBoard extension.

In [ ]:
%load_ext tensorboard

# removes logs from previous runs
!rm -rf ./logs/ 

TensorBoard si ukážeme na malé neuronové síti 🧠, jejímž úkolem bude klasifikovat obrázky z datasetu [MNIST](https://en.wikipedia.org/wiki/MNIST_database). Je to dataset černobílých obrázků ručně psaných číslic ✍️. Každý obrázek má 28x28 pixelů a barva každého pixelu je reprezentována jedním číslem od 0 do 255.

O práci s obrázky jsme si povídali na předminulé přednášce.☝️

In [ ]:
mnist = tf.keras.datasets.mnist

(X_train, y_train),(X_test, y_test) = mnist.load_data()

# normalizes image data
X_train = X_train / 255.0
X_test = X_test / 255.0

Příznaky jsme normalizovali, takže se hodnoty pixelů pohybují v intervalu od 0 do 1.

Nyní se můžeme posunout k vytvoření neuronové sítě. Není třeba, abyste rozuměli každému detailu v tomto kódu. O neuronových sítích se budete učit více v jiných předmětech 😉. 

Zkusme si práci s balíčkem *tensorflow* porovnat se *scikit-learn*, který už znáte.

Ve *scikit-learn* model vytvoříme pomocí jeho konstruktoru. Například rozhodovací strom pro klasifikaci vytvoříme voláním `DecisionTreeClassifier()`. V *tensorflow* jsme model vytvořili pomocí volání `keras.models.Sequential`, kde jsme jako parametr zadali pole vrstev, které má neuronová síť obsahovat.

Neuronová síť potřebuje pár informací navíc, kterými jsou hlavně ztrátová funkce a evaluační metrika. Ztrátová funkce se používá při optimalizaci vah a metrika zase na výběr nejlepšího modelu 💪. Tyto údaje se zadávají ve funkci `compile`.

Ve *scikit-learn* jsme model natrénovali tak, že jsme zavolali jeho funkci `fit` 🏋️‍♂️. Úplně analogicky to funguje také v *tensorflow*.

In [ ]:
# model architecture
def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# provides loss function, metrics and optimizer
def compile_model(model):
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
def create_log_dir():
    return "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

model = create_model()
compile_model(model)
log_dir = create_log_dir()

Aby TensorBoard mohl fungovat, potřebujeme mu vygenerovat nějaká data (logy) 📚, která bude vykreslovat. V předchozí buňce jsme si vytvořili složku, do které se logy budou ukládat.

Nyní vytvoříme callback. Tensorflow callback-y jsou funkce nebo bloky kódu, které se spustí v nějakých specifických momentech během trénování modelu. Můžeme pomocí nich měnit hyperparametry, ukončit trénování nebo právě to, co potřebujeme – zalogovat data pro TensorBoard. K logování slouží [`tf.keras.callbacks.TensorBoard`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) callback.

Aby vše fungovalo, musíme callback prodat modelu ve funkci [`Model.fit`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit).

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1) # 1 stands for each epoch

model.fit(
    x=X_train, 
    y=y_train, 
    epochs=10, 
    verbose=0,
    validation_data=(X_test, y_test),
    # this is important
    callbacks=[tensorboard_callback]
)

Poté, co doběhne trénování, si můžeme zkusit zobrazit výsledky. TensorBoard lze zobrazit buď přímo v notebooku pomocí příkazu v následující buňce nebo v prohlížeči 🌐. V případě zobrazení v prohlížeči je třeba zadat příkaz `tensorboard --logdir logs` do terminálu a do prohlížeče zkopírovat localhost URL, kterou v terminálu uvidíte.

In [ ]:
# runs tensorboard with data in directory named logs
%tensorboard --logdir logs

Jednotlivé záložky v horním navigačním baru se nazývají dashboardy:

* Dashboard **Scalars** ukazuje, jak se v každé epoše měnila metrika a ztráta (loss). Tento dashboard lze použít i pro sledování jiných skalárních hodnot.
* Dashboard **Graphs** slouží k vizualizaci architektury modelu. Díky tomu se můžeme ujistit, že při sestavování modelu nedošlo k nějaké chybě a vypadá tak, jak bychom čekali.

TensorBord nabízí mnohem více vizualizací. Vždy se zobrazí jen ty vizualizace, ke kterým jsme během trénování logovali potřebná data. Všechny typy vizualizací, které jsou dostupné, si můžete prohlédnout kliknutím na dropdown tlačítko `inactive` v horním navigačním panelu.

## ☝️ Scalars
Metriky jsou ve strojovém učení velmi důležité a jejich správná interpretace může například pomoci odhalit přeučení nebo ukázat, že model trénujeme zbytečně dlouho.

Dashboard Scalars umožňuje zobrazit vývoj metrik napříč epochami. V základní podobě se sbírají data pro metriky a ztrátovou funkci, které jsme uvedli při kompilaci modelu. Data se sbírají pro trénovací a validační množinu.

Aby se začala logovat data pro metriky v TensorBoardu, je třeba provést následující:

* 📁 specifikovat složku, kde se budou logy ukládat
* 🤙 vytvořit [TensorBoard callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard)
* 🏋️‍♂️ prodat TensorBoard callback modelu při volání [`Model.fit()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit)

Toto vše jsme již udělali v kódu výše, ale pro zopakování:
```python
# specify log directory
log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# create callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

model.fit(
    ...
    # pass the callback to the model
    callbacks=[tensorboard_callback]
)
````

Po natrénování modelu TensorBoard načte data z logovací složky. V našem případě je root složka `logs`. V této složce vzniknou nové složky, jejichž název bude shodný s timestampem v době jejich vytvoření. Díky tomuto přístupu můžeme pak v TensorBoardu porovnat data z více běhů. Na výběr podmnožiny běhů, které nás zajímají, slouží sekce **Runs** 🏃 v levém panelu.

### 📚 Logování vlastních hodnot
Logování vlastních hodnot si ukážeme na příkladu logování tzv. learning rate (míra učení). Budeme postupovat následovně:
* vytvoříme file writer pomocí `tf.summary.create_file_writer()`
* zadefinujeme funkci, která nám na základě čísla epochy vrátí learning rate
  * v této funkci použijeme tf.summary.scalar(), která zaloguje nově vypočtenou hodnotu při každém volání funkce
* vytvoříme callback (v tomto případě LearningRateScheduler callback)
* callback prodáme modelu při volání `Model.fit`

Obecně při logování vlastních hodnot je třeba vždy vytvořit file writer a poté použít `tf.summary.scalar()` k zalogování nové hodnoty.

In [ ]:
log_dir = create_log_dir()
file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
file_writer.set_as_default()

def compute_learning_rate(epoch):
    learning_rate = 0.02
    if epoch > 3:
        learning_rate = 0.01
    if epoch > 7:
        learning_rate = 0.005

    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

lr_callback = keras.callbacks.LearningRateScheduler(compute_learning_rate)

In [ ]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)

model = create_model()
compile_model(model)

model.fit(
    x=X_train, 
    y=y_train, 
    epochs=10,
    verbose=0,
    validation_data=(X_test, y_test),
    # this is important
    callbacks=[tensorboard_callback, lr_callback]
)

file_writer.close()

## 📊 Graphs
V dashboardu Graphs se defaultně (v levé liště je jako tag zvolen *Default*) zobrazuje operation-level (zkráceně op-level) graf.

### 💭 Konceptuální graf
Jednodušší k pochopení je konceptuální graf, který vizualizuje jen Keras model. Zobrazíme ho tak, že jako tag zvolíme *keras*. V našem případě se pak zobrazí jeden uzel, který expanduje, pokud na něj dvakrát klikneme. V expandované formě graf zobrazuje jednotlivé vrstvy Keras modelu a šipky ukazují směr proudění dat. První vrstva reprezentuje vstupní data a poté následují 4 vrstvy, které odpovídají naší definici modelu:

```python
tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(10, activation='softmax')
])
````

![conceptual_graph](img/conceptual_graph.png)

Více informací k jednotlivým vrstvám grafu lze zobrazit tak, že na ně klikneme.

### 🔬 Operational-level graf
Op-level graf zobrazuje každou operaci, která se s daty děje. Graf tedy obsahuje Keras model, ale i všechny ostatní operace jako například výpočet metrik a podobně. Pozor na to, že graf je oproti kódu obrácen vzhůru nohama – flow dat jde zdola nahoru.

Výsledná vizualizace je často velká, lze s ní manipulovat:
* přibližuje a oddaluje se pomocí scrollování,
* posouvat se dá kliknutím a potáhnutím,
* vrcholy jsou expandovatelné/kolapsovatelné pomocí dvojkliku,
* jeden klik na vrchol zobrazí jeho metadata.

## 🔗 TensorBoard.dev - hosting ML výsledků

[TensorBoard.dev](https://tensorboard.dev) umožňuje zdarma uploadnout TensorBoard logy a vygenerovat link, který lze sdílet s kýmkoliv.

Pro použití TensorBoard.dev stačí zadat do terminálu tento příkaz (name a description jsou volitelné).

```shell
tensorboard dev upload \
  --logdir logs/fit \
  --name "BI-VIZ TensorBoard test" \
  --description "Visualizations for neural network trained on MNIST dataset" \
  --one_shot
```